# Входные данные
- Две функции, принимающие загаданное число и возвращающие число попыток.
- Функция для оценки среднего кол-ва попыток на выборке из 1000 случайных чисел от 1 до 100.

# Требуется
- Написать функцию, которая будет угадывать число за меньшее кол-во попыток.

In [35]:
import numpy as np
import re

def game_core_v1(number, low_number=1, high_number=100):
    '''Просто угадываем на random, никак не используя информацию о больше или меньше.
       Функция принимает загаданное число, интервал (по умолчанию от 1 до 100) и возвращает число попыток
    '''    
    count = 0
    
    while True:
        count += 1
        predict = np.random.randint(low_number, high_number + 1) # предполагаемое число
        if number == predict: 
            return count 
        
def game_core_v2(number, low_number=1, high_number=100):
    '''Сначала устанавливаем любое random число, а потом уменьшаем или увеличиваем его 
    в зависимости от того, больше оно или меньше нужного.
    Функция принимает загаданное число, интервал (по умолчанию от 1 до 100) и возвращает число попыток
    '''
    count = 1
    predict = np.random.randint(low_number, high_number + 1)
    
    while number != predict:
        count += 1
        if number > predict: 
            predict += 1
        elif number < predict: 
            predict -= 1
    return(count) 



def score_game(game_core, low_number=1, high_number=100):
    '''Запускаем игру 1000 раз, чтобы узнать, как быстро игра угадывает число
    '''
    pattern = re.compile('v\d+')
    count_ls = []
    np.random.seed(1)  # фиксируем RANDOM SEED, чтобы ваш эксперимент был воспроизводим!
    random_array = np.random.randint(low_number, high_number + 1, size=(1000))
    
    for number in random_array:
        count_ls.append(game_core(number, low_number, high_number))
        
    score = int(np.mean(count_ls))
    version = pattern.findall(str(game_core))[0]
    print(f"Алгоритм {version} угадывает число в среднем за {score} попыток")
    return

In [36]:
game_core_v2(100)

67

## Решение через метод половинного деления

_Дополнительно разрешаем указывать произвольный диапазон_

In [58]:
def game_core_v30(number, low_number=1, high_number=100):
    '''Функция принимает загаданное число в заданном диапазоне (по умолчанию от 1 до 100).
    Через метод половинного деления компьютер угадывает введеное число.
    Функция возвращает количество попыток.
    '''
    if low_number >= high_number or number < low_number or number > high_number:  
        print('Неверное число или диапазон')  # валидация входных данных 
        raise ValueError('Incorrect argumets') # кидаем ошибку    
    else:
        count = 1 
        predict = (low_number+high_number) // 2 # ищем середину интервала между введенными значениzvb

        while number != predict:  
            count += 1    
            if predict < number: 
                if predict != number - 1:
                    low_number = predict + 1  # сокращаем интервал в два раза для ускорения поиска
                    predict = (low_number+high_number) // 2 # ищем середину нового интервала
                else:
                    predict += 1  # число совпало с верхней границей интервала          
            elif predict > number: 
                if predict != number + 1:
                    high_number = predict - 1
                    predict = (low_number+high_number) // 2 # ищем середину нового интервала
                else:
                    predict -= 1  # число совпало с нижней границей интевала                  
        return(count)  # возращаем количество попыток
    
print(game_core_v30(65))
print(game_core_v30(100, 1, 100))
print(game_core_v30(30, high_number=500))

5
7
5


Та же логика, но реализация через списки

In [60]:
def game_core_v20(number, low_number=1, high_number=100):
    '''Функция принимает загаданное число в заданном диапазоне (по умолчанию от 1 до 100).
    Через метод половинного деления компьютер угадывает введеное число.
    Функция возвращает количество попыток.
    '''
    if low_number > high_number or number < low_number or number > high_number:
        print('Неверное число или диапазон') 
        raise ValueError('Incorrect argumets')
    else:
        count = 1
        list_of_numbers = [x for x in range(low_number, high_number + 1)]  # числовой ряд в заданном интервале
        middle = len(list_of_numbers) // 2  # находим середину числового ряда
        predict = list_of_numbers[middle]  # делаем попытку угадать

        while number != predict:  # пока не угадаем число, сокращаем интервал пополам
            count += 1
            if number > predict: 
                list_of_numbers = list_of_numbers[middle:]
            elif number < predict: 
                list_of_numbers = list_of_numbers[:middle]
            middle = len(list_of_numbers) // 2
            predict = list_of_numbers[middle]        
        return(count) # возвращаем количество попыток

print(game_core_v20(70))
print(game_core_v20(100, 100, 500))
print(game_core_v20(100, high_number=500))

6
9
8


### Сравнение разных алгоритмов 
Метод половинного деления находит число на порядок быстрее на дефолтном интервале

In [61]:
score_game(game_core_v1)
score_game(game_core_v2)
score_game(game_core_v20)
score_game(game_core_v30)

Алгоритм v1 угадывает число в среднем за 101 попыток
Алгоритм v2 угадывает число в среднем за 33 попыток
Алгоритм v20 угадывает число в среднем за 5 попыток
Алгоритм v30 угадывает число в среднем за 4 попыток


А на интервале от 1 до 1000 поиск быстрее на два порядка

In [62]:
score_game(game_core_v1, 1, 1000)
score_game(game_core_v2, 1, 1000)
score_game(game_core_v20, 1, 1000)
score_game(game_core_v30, 1, 1000)

Алгоритм v1 угадывает число в среднем за 954 попыток
Алгоритм v2 угадывает число в среднем за 355 попыток
Алгоритм v20 угадывает число в среднем за 9 попыток
Алгоритм v30 угадывает число в среднем за 8 попыток
